# De Novo Protein Design Workflow using NIMs (skipped AlphaFold2 prediction step)

Workflow for creating de novo protein binders using NVIDIA Inference Microservices (NIMs). The NIMs are deployed via NVIDIA-hosted cloud GPU platform.

In this workflow, we take an alternative approach to bypass having to perform Alphafold2 structural prediction (step #1) on NVIDIA cloud GPU. Instead, we provide it a pre-computed protein structure (.PDB) generated on [AlphaFold2 colab](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=R_AH6JSXaeb2) with our specified protein sequence. 

Given the AlphaFold2 .PDB as input, protein backbones are then generated with **RFDiffusion**, sequences are generated with **ProteinMPNN**, and finally complex structures are predicted with **AlphaFold2-multimer**. 

# Section A: Set up cloud server, and run docker containers

### Pre-requisites

Software requirements: Python 3.11+

Hardware requirements: at least 3 x GPU, 47 GiB GPU memory, 1.3TB GB SSD drive space, 60GiB RAM,24 CPU
- RFdiffusion runs on 1 x GPU, ≥12 GiB GPU memory, 15GB free SSD drive space
- ProteinMPNN runs on 1 x GPU, ≥3 GiB GPU memory, 10GB free SSD drive space
- AlphaFold-Multimer runs on 1 x GPU, ≥32 GiB GPU memory, 512GB free SSD drive space, 64GB RAM, 24 CPU 

### Ensure that you have these files ready:

- `protein-binder-design_v3.ipynb` uploaded to a [public Github repo](https://github.com/Keonapang/generative-protein-binder-design/blob/main/src/protein-binder-design.ipynb)
- `docker-compose.yaml` (3MB) from the original [BioNeMo repo](https://github.com/NVIDIA-BioNeMo-blueprints/generative-protein-binder-design/blob/main/deploy/docker-compose.yaml)
- `cycle1_alphafold2_output.pdb` (80KB) pre-computed on [AlphaFold2 colab](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=R_AH6JSXaeb2)

### Deploy NIM on NVIDIA-hosted API (cloud GPU)

1. Go to [brev.nvidia](https://brev.nvidia.com/) and create a new **'Launchable'** with the following settings:
    - **Compute**: A100 (80GiB GPU memory) 4 GPUs x 48 CPUs | 480GiB | 128GiB RAM ($7.92/hr)
    - **Container**: use VM-mode (installing jupyter notebook)
    - **File**: Link to blueprint's [jupyter notebook](https://github.com/Keonapang/generative-protein-binder-design/blob/main/src/protein-binder-design_v3.ipynb)

2. Then, click **"Launch"** and "Go to Instance Page". Wait 15-20 minutes for the cloud server to deploy.

3. When the server is ready, **upload** 2 files (you can find them on this repo): 
    - `./deploy/docker-compose_v3.yaml` sets up the Docker images, networks, and complex dependencies required by each NIM (NVIDIA Inference Module). All NIMs are packaged as containers for secure and easy deployment.
    - `./docs/cycle1_alphafold2_output.pdb` from AlphaFold2 on Colab notebook

4.  Copy codes from brev.nvidias Instance Page > under "Access" tab and run on Terminal window of your local computer:

```bash
    brew install brevdev/homebrew-/brev && brev login --token <****> # Install the CLI
    brev shell <instance-name< # find instance-name on brev.nvidias # Open a terminal locally

    # update
    sudo apt-get install -y docker-compose
    sudo apt install python3.11
```
5. Run on terminal. Ensure that you've [generated](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/clara/containers/bionemo-framework) a **NGC Personal Key**. 

```bash
    export NGC_CLI_API_KEY=<enter-key> # enter personal API key
    docker login nvcr.io --username='$oauthtoken' --password="${NGC_CLI_API_KEY}"

    ## Create the nim cache directory to download any model data to your local/server disk 
    mkdir -p ~/.cache/nim
    chmod -R 777 ~/.cache/nim    ## Make it writable by the NIM
    export HOST_NIM_CACHE=~/.cache/nim

    # Run Docker compose
    docker compose 
```


Running **Docker Compose** takes 20-25mins total. When the containers start, they will begin by pulling the models for each NIM. The terminal will look like:

```bash
    [+] Running 4/4
    ✔ Network protein-binder-design_default                 Created          0.1s 
    ✔ Container protein-binder-design-alphafold-multimer-1  Started          6.3s 
    ✔ Container protein-binder-design-rfdiffusion-1         Started          6.2s 
    ✔ Container protein-binder-design-proteinmpnn-1         Started          6.2s 
```

6. **Finally, check the status** of the four running NIMS with the command:

```bash
    curl localhost:8081/v1/health/ready # alphafold2
    curl localhost:8082/v1/health/ready # RFdiffusion
    curl localhost:8083/v1/health/ready # Protein MPNN
    curl localhost:8084/v1/health/ready # alphafold2-multimer

    # check what dockers are running
    docker container ls
    docker container logs <CONTAINER-ID> # get CONTAINER-ID from command above

    ls -lah     # print list of directories 
    df -h       # print storage space
    
    cd nim # ngc  nim  warp

    docker run --rm --runtime=nvidia --gpus all ubundu nvidia-smi
    pip install requests
```

# Section B: Start up the NIMs
First, install prerequisites

In [ ]:
import json
import os
import requests
from enum import StrEnum, Enum # must be Python 3.11+ 
from typing import Tuple, Dict, Any, List
from pathlib import Path

We need to start up the NIMs. While this will return quickly, it will take some time for the models to download (**3-6 hours on a 1000mbps internet connection**).

In [ ]:
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY") or input("Paste Run Key: ")

In [ ]:
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {NVIDIA_API_KEY}",
    "poll-seconds": "900"
}
NIM_HOST_URL_BASE = "http://localhost"
# 3 different endpoints for the models
class NIM_PORTS(Enum):
    RFDIFFUSION_PORT = 8082
    PROTEINMPNN_PORT = 8083
    AF2_MULTIMER_PORT = 8084

class NIM_ENDPOINTS(StrEnum):
    RFDIFFUSION =  "biology/ipd/rfdiffusion/generate"
    PROTEINMPNN =  "biology/ipd/proteinmpnn/predict"
    AF2_MULTIMER = "protein-structure/alphafold2/multimer/predict-structure-from-sequences"

def query_nim(
            payload: Dict[str, Any],
            nim_endpoint: str,
            headers: Dict[str, str] = HEADERS,
            base_url: str = "http://localhost",
            nim_port: int = 8080,
            echo: bool = False) -> Tuple[int, Dict]:
    function_url = f"{base_url}:{nim_port}/{nim_endpoint}"
    if echo:
        print("*"*80)
        print(f"\tURL: {function_url}")
        print(f"\tPayload: {payload}")
        print("*"*80)
    response = requests.post(function_url,
                            json=payload,
                            headers=headers)
    if response.status_code == 200:
        return response.status_code, response.json()
    else:
        raise Exception(f"Error: response returned code [{response.status_code}], with text: {response.text}")

def check_nim_readiness(nim_port: NIM_PORTS,
                        base_url: str = NIM_HOST_URL_BASE,
                        endpoint: str = "v1/health/ready") -> bool:
    """
    Return true if a NIM is ready.
    """
    try:
        response = requests.get(f"{base_url}:{nim_port}/{endpoint}")
        d = response.json()
        if "status" in d:
            if d["status"] == "ready":
                return True
        return False
    except Exception as e:
        print(e)
        return False

def get_reduced_pdb(pdb_id: str, rcsb_path: str = None) -> str:
    pdb = Path(pdb_id)
    if not pdb.exists() and rcsb_path is not None:
        pdb.write_text(requests.get(rcsb_path).text)
    lines = filter(lambda line: line.startswith("ATOM"), pdb.read_text().split("\n"))
    return "\n".join(list(lines))

class ExampleRequestParams:
    def __init__(self,
                target_sequence: str,
                contigs: str, 
                hotspot_res: List[str],
                input_pdb_chains: List[str],
                ca_only: bool,
                use_soluble_model: bool,
                sampling_temp: List[float],
                diffusion_steps: int = 15,
                num_seq_per_target: int = 20):
        self.target_sequence = target_sequence
        self.contigs = contigs
        self.hotspot_res = hotspot_res
        self.input_pdb_chains = input_pdb_chains
        self.ca_only = ca_only
        self.use_soluble_model = use_soluble_model
        self.sampling_temp = sampling_temp
        self.diffusion_steps = diffusion_steps
        self.num_seq_per_target = num_seq_per_target

Test whether each NIM is up and running using our check_nim_readiness function. 

In [ ]:
status = check_nim_readiness(NIM_PORTS.RFDIFFUSION_PORT.value)
print(f"RFDiffusion ready: {status}")
status = check_nim_readiness(NIM_PORTS.PROTEINMPNN_PORT.value)
print(f"ProteinMPNN ready: {status}")
status = check_nim_readiness(NIM_PORTS.AF2_MULTIMER_PORT.value)
print(f"AlphaFold2-multimer ready: {status}")

# Section C: Peptide design
### Create 8 peptide designs for each iteration

Specifications:
- 8 peptides; 10-20 amino acid residues long
- peptides are distanced 30-40 AA apart
- bewteen cycle 1 and cycle 2, there is about a 1000AA gap 

These will exhibit different runtimes and resource utilizations:
- 600 AA length `target_sequence` requires 4 GPUs with > 40GB of VRAM each, or 12 mins on H100, or ~25 mins on 1 A6000 GPU
- 1,281 AA length `target_sequence` requires 4 GPUs with > 80GB of VRAM each

In [ ]:
cycle1 = ExampleRequestParams(
    target_sequence="MDPPRPALLALLALPALLLLLLAGARAEEEMLENVSLVCPKDATRFKHLRKYTYNYEAESSSGVPGTADSRSATRINCKVELEVPQLCSFILKTSQCTLKEVYGFNPEGKALLKKTKNSEEFAAAMSRYELKLAIPEGKQVFLYPEKDEPTYILNIKRGIISALLVPPETEEAKQVLFLDTVYGNCSTHFTVKTRKGNVATEISTERDLGQCDRFKPIRTGISPLALIKGMTRPLSTLISSSQSCQYTLDAKRKHVAEAICKEQHLFLPFSYKNKYGMVAQVTQTLKLEDTPKINSRFFGEGTKKMGLAFESTKSTSPPKQAEAVLKTLQELKKLTISEQNIQRANLFNKLVTELRGLSDEAVTSLLPQLIEVSSPITLQALVQCGQPQCSTHILQWLKRVHANPLLIDVVTYLVALIPEPSAQQLREIFNMARDQRSRATLYALSHAVNNYHKTNPTGTQELLDIANYLMEQIQDDCTGDEDYTYLILRVIGNMGQTMEQLTPELKSSILKCVQSTKPSLMIQKAAIQALRKMEPKDKDQEVLLQTFLDDASPGDKRLAAYLMLMRSPSQADINKIVQILPWEQNEQVKNFVASHIANILNSEELDIQDLKKLVKEALKESQLPTVMDFRKFSRNYQLYKSVSLPSLDPASAKIEGNLIFDPNNYLPKESMLKTTLTAFGFASADLIEIGLEGKGFEPTLEALFGKQGFFPDSVNKALYWVNGQVPDGVSKVLVDHFGYTKDDKHEQDMVNGIMLSVEKLIKDLKSKEVPEARAYLRILGEELGFASLHDLQLLGKLLLMGARTLQGIPQMIGEVIRKGSKNDFFLHYIFMENAFELPTGAGLQLQISSSGVIAPGAKAGVKLEVANMQAELVAKPSVSVEFVTNMGIIIPDFARSGVQMNTNFFHESGLEAHVALKAGKLKFIIPSPKRPVKLLSGGNTLHLVSTTKTEVIPPLIENRQSWSVCKQVFPGLNYCTSGAYSNASSTDSASYYPLTGDTRLELELRPTGEIEQYSVSATYELQREDRALVDTLKFVTQAEGAKQTEATMTFKYNRQSMTLSSEVQIPDFDVDLGTILRVNDESTEGKTSYRLTLDIQNKKITEVALMGHLSCDTKEERKIKGVISIPRLQAEARSEILAHWSPAKLLLQMDSSATAYGSTVSKRVAWHYDEEKIEFEWNTGTNVDTKKMTSNFPVDLSDYPKSLHMYANRLLDHRVPQTDMTFRHVGSKLIVAMSSWLQKASGSLPYTQTLQDHLNSLKEFNLQNMGLPDFHIPENLFLKSDGRVKYTLNKN",
    contigs="A400-600/0 15-25",  # Region A400-A600, peptides 15-25 residues long
    hotspot_res=[], 
    input_pdb_chains=[], # [Optional] default is to design for all chains in the protein
    ca_only=False, # [Optional]  CA-only model helps to address specific needs in protein design where focusing on the alpha carbon (CA)
    use_soluble_model=True, 
    sampling_temp=[0.2], # (range: 0 - 1) adjust the probability values for the 20 amino acids at each position, controls the diversity of the design outcomes
    diffusion_steps=30, # 15-30 steps are recommended for protein design tasks
    num_seq_per_target=4  # Generate 4 binders
)

cycle2 = ExampleRequestParams(
    target_sequence="GKIDFLNNYALFLSPSAQQASWQVSARFNQYKYNQNFSAGNNENIMEAHVGINGEANLDFLNIPLTIPEMRLPYTIITTPPLKDFSLWEKTGLKEFLKTTKQSFDLSVKAQYKKNKHRHSITNPLAVLCEFISQSIKSFDRHFEKNRNNALDFVTKSYNETKIKFDKYKAEKSHDELPRTFQIPGYTVPVVNVEVSPFTIEMSAFGYVFPKAVSMPSFSILGSDVRVPSYTLILPSLELPVLHVPRNLKLSLPDFKELCTISHIFIPAMGNITYDFSFKSSVITLNTNAELFNQSDIVAHLLSSSSSVIDALQYKLEGTTRLTRKRGLKLATALSLSNKFVEGSHNSTVSLTTKNMEVSVATTTKAQIPILRMNFKQELNGNTKSKPTVSSSMEFKYDFNSSMLYSTAKGAVDHKLSLESLTSYFSIESSTKGDVKGSVLSREYSGTIASEANTYLNSKSTRSSVKLQGTSKIDDIWNLEVKENFAGEATLQRIYSLWEHSTKNHLQLEGLFFTNGEHTSKATLELSPWQMSALVQVHASQPSSFHDFPDLGQEVALNANTKNQKIRWKNEVRIHSGSFQSQVELSNDQEKAHLDIAGSLEGHLRFLKNIILPVYDKSLWDFLKLDVTTSIGRRQHLRVSTAFVYTKNPNGYSFSIPVKVLADKFIIPGLKLNDLNSVLVMPTFHVPFTDLQVPSCKLDFREIQIYKKLRTSSFALNLPTLPEVKFPEVDVLTKYSQPEDSLIPFFEITVPESQLTVSQFTLPKSVSDGIAALDLNAVANKIADFELPTIIVPEQTIEIPSIKFSVPAGIVIPSFQALTARFEVDSPVYNATWSASLKNKADYVETVLDSTCSSTVQFLEYELNVLGTHKIEDGTLASKTKGTFAHRDFSAEYEEDGKYEGLQEWEGKAHLNIKSPAFTDLHLRYQKDKKGISTSAASPAVGTVGMDMDEDDDFSKWNFYYSPQSSPDKKLTIFKTELRVRESDEETQIKVNWEEEAASGLLTSLKDNVPKATGVLYDYVNKYHWEHTGLTLREVSSKLRRNLQNNAEWVYQGAIRQIDDIDVRFQKAASGTTGTYQEWKDKAQNLYQELLTQEGQASFQGLKDNVFDGLVRVTQEFHMKVKHLIDSLIDFLNFPRFQFPGKPGIYTREELCTMFIREVGTVLSQVYSKVHNGSEILFSYFQDLVITLPFELRKHKLIDV",
    contigs="A100-300/0 15-25",  # Region A400-A600, peptides 15-25 residues long
    hotspot_res=[], 
    input_pdb_chains=[], # [Optional] default is to design for all chains in the protein
    ca_only=False, # [Optional]  CA-only model helps to address specific needs in protein design where focusing on the alpha carbon (CA)
    use_soluble_model=True, 
    sampling_temp=[0.2], # (range: 0 - 1) adjust the probability values for the 20 amino acids at each position, controls the diversity of the design outcomes
    diffusion_steps=30, # 15-30 steps are recommended for protein design tasks
    num_seq_per_target=4  # Generate 4 binders
)

###### Cycle 1: first-half of the ApoB protein ######
cycle1A = ExampleRequestParams(
    target_sequence="MDPPRPALLALLALPALLLLLLAGARAEEEMLENVSLVCPKDATRFKHLRKYTYNYEAESSSGVPGTADSRSATRINCKVELEVPQLCSFILKTSQCTLKEVYGFNPEGKALLKKTKNSEEFAAAMSRYELKLAIPEGKQVFLYPEKDEPTYILNIKRGIISALLVPPETEEAKQVLFLDTVYGNCSTHFTVKTRKGNVATEISTERDLGQCDRFKPIRTGISPLALIKGMTRPLSTLISSSQSCQYTLDAKRKHVAEAICKEQHLFLPFSYKNKYGMVAQVTQTLKLEDTPKINSRFFGEGTKKMGLAFESTKSTSPPKQAEAVLKTLQELKKLTISEQNIQRANLFNKLVTELRGLSDEAVTSLLPQLIEVSSPITLQALVQCGQPQCSTHILQWLKRVHANPLLIDVVTYLVALIPEPSAQQLREIFNMARDQRSRATLYALSHAVNNYHKTNPTGTQELLDIANYLMEQIQDDCTGDEDYTYLILRVIGNMGQTMEQLTPELKSSILKCVQSTKPSLMIQKAAIQALRKMEPKDKDQEVLLQTFLDDASPGDKRLAAYLMLMRSPSQADINKIVQILPWEQNEQVKNFVASHIANILNSEELDIQDLKKLVKEALKESQLPTVMDFRKFSRNYQLYKSVSLPSLDPASAKIEGNLIFDPNNYLPKESMLKTTLTAFGFASADLIEIGLEGKGFEPTLEALFGKQGFFPDSVNKALYWVNGQVPDGVSKVLVDHFGYTKDDKHEQDMVNGIMLSVEKLIKDLKSKEVPEARAYLRILGEELGFASLHDLQLLGKLLLMGARTLQGIPQMIGEVIRKGSKNDFFLHYIFMENAFELPTGAGLQLQISSSGVIAPGAKAGVKLEVANMQAELVAKPSVSVEFVTNMGIIIPDFARSGVQMNTNFFHESGLEAHVALKAGKLKFIIPSPKRPVKLLSGGNTLHLVSTTKTEVIPPLIENRQSWSVCKQVFPGLNYCTSGAYSNASSTDSASYYPLTGDTRLELELRPTGEIEQYSVSATYELQREDRALVDTLKFVTQAEGAKQTEATMTFKYNRQSMTLSSEVQIPDFDVDLGTILRVNDESTEGKTSYRLTLDIQNKKITEVALMGHLSCDTKEERKIKGVISIPRLQAEARSEILAHWSPAKLLLQMDSSATAYGSTVSKRVAWHYDEEKIEFEWNTGTNVDTKKMTSNFPVDLSDYPKSLHMYANRLLDHRVPQTDMTFRHVGSKLIVAMSSWLQKASGSLPYTQTLQDHLNSLKEFNLQNMGLPDFHIPENLFLKSDGRVKYTLNKN",
    contigs="A400-440/0 15-25", # 15-25 AA
    hotspot_res=[], 
    input_pdb_chains=[], # [Optional] default is to design for all chains in the protein
    ca_only=False, # [Optional]  CA-only model helps to address specific needs in protein design where focusing on the alpha carbon (CA)
    use_soluble_model=True, # bloodstream
    sampling_temp=[0.2], # (range: 0 - 1) adjust the probability values for the 20 amino acids at each position, controls the diversity of the design outcomes
    diffusion_steps=30, # 15-30 steps are recommended for protein design tasks
    num_seq_per_target=1 # [Optional] Default is 1.Number of sequences to generate per target protein structure. 
)
cycle1B = ExampleRequestParams(
    target_sequence="MDPPRPALLALLALPALLLLLLAGARAEEEMLENVSLVCPKDATRFKHLRKYTYNYEAESSSGVPGTADSRSATRINCKVELEVPQLCSFILKTSQCTLKEVYGFNPEGKALLKKTKNSEEFAAAMSRYELKLAIPEGKQVFLYPEKDEPTYILNIKRGIISALLVPPETEEAKQVLFLDTVYGNCSTHFTVKTRKGNVATEISTERDLGQCDRFKPIRTGISPLALIKGMTRPLSTLISSSQSCQYTLDAKRKHVAEAICKEQHLFLPFSYKNKYGMVAQVTQTLKLEDTPKINSRFFGEGTKKMGLAFESTKSTSPPKQAEAVLKTLQELKKLTISEQNIQRANLFNKLVTELRGLSDEAVTSLLPQLIEVSSPITLQALVQCGQPQCSTHILQWLKRVHANPLLIDVVTYLVALIPEPSAQQLREIFNMARDQRSRATLYALSHAVNNYHKTNPTGTQELLDIANYLMEQIQDDCTGDEDYTYLILRVIGNMGQTMEQLTPELKSSILKCVQSTKPSLMIQKAAIQALRKMEPKDKDQEVLLQTFLDDASPGDKRLAAYLMLMRSPSQADINKIVQILPWEQNEQVKNFVASHIANILNSEELDIQDLKKLVKEALKESQLPTVMDFRKFSRNYQLYKSVSLPSLDPASAKIEGNLIFDPNNYLPKESMLKTTLTAFGFASADLIEIGLEGKGFEPTLEALFGKQGFFPDSVNKALYWVNGQVPDGVSKVLVDHFGYTKDDKHEQDMVNGIMLSVEKLIKDLKSKEVPEARAYLRILGEELGFASLHDLQLLGKLLLMGARTLQGIPQMIGEVIRKGSKNDFFLHYIFMENAFELPTGAGLQLQISSSGVIAPGAKAGVKLEVANMQAELVAKPSVSVEFVTNMGIIIPDFARSGVQMNTNFFHESGLEAHVALKAGKLKFIIPSPKRPVKLLSGGNTLHLVSTTKTEVIPPLIENRQSWSVCKQVFPGLNYCTSGAYSNASSTDSASYYPLTGDTRLELELRPTGEIEQYSVSATYELQREDRALVDTLKFVTQAEGAKQTEATMTFKYNRQSMTLSSEVQIPDFDVDLGTILRVNDESTEGKTSYRLTLDIQNKKITEVALMGHLSCDTKEERKIKGVISIPRLQAEARSEILAHWSPAKLLLQMDSSATAYGSTVSKRVAWHYDEEKIEFEWNTGTNVDTKKMTSNFPVDLSDYPKSLHMYANRLLDHRVPQTDMTFRHVGSKLIVAMSSWLQKASGSLPYTQTLQDHLNSLKEFNLQNMGLPDFHIPENLFLKSDGRVKYTLNKN",
    contigs="A450-490/0 15-25",  # 15 AA long; A460-480/0 15-15
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=30, 
    num_seq_per_target=1 
)
cycle1C = ExampleRequestParams(
    target_sequence="MDPPRPALLALLALPALLLLLLAGARAEEEMLENVSLVCPKDATRFKHLRKYTYNYEAESSSGVPGTADSRSATRINCKVELEVPQLCSFILKTSQCTLKEVYGFNPEGKALLKKTKNSEEFAAAMSRYELKLAIPEGKQVFLYPEKDEPTYILNIKRGIISALLVPPETEEAKQVLFLDTVYGNCSTHFTVKTRKGNVATEISTERDLGQCDRFKPIRTGISPLALIKGMTRPLSTLISSSQSCQYTLDAKRKHVAEAICKEQHLFLPFSYKNKYGMVAQVTQTLKLEDTPKINSRFFGEGTKKMGLAFESTKSTSPPKQAEAVLKTLQELKKLTISEQNIQRANLFNKLVTELRGLSDEAVTSLLPQLIEVSSPITLQALVQCGQPQCSTHILQWLKRVHANPLLIDVVTYLVALIPEPSAQQLREIFNMARDQRSRATLYALSHAVNNYHKTNPTGTQELLDIANYLMEQIQDDCTGDEDYTYLILRVIGNMGQTMEQLTPELKSSILKCVQSTKPSLMIQKAAIQALRKMEPKDKDQEVLLQTFLDDASPGDKRLAAYLMLMRSPSQADINKIVQILPWEQNEQVKNFVASHIANILNSEELDIQDLKKLVKEALKESQLPTVMDFRKFSRNYQLYKSVSLPSLDPASAKIEGNLIFDPNNYLPKESMLKTTLTAFGFASADLIEIGLEGKGFEPTLEALFGKQGFFPDSVNKALYWVNGQVPDGVSKVLVDHFGYTKDDKHEQDMVNGIMLSVEKLIKDLKSKEVPEARAYLRILGEELGFASLHDLQLLGKLLLMGARTLQGIPQMIGEVIRKGSKNDFFLHYIFMENAFELPTGAGLQLQISSSGVIAPGAKAGVKLEVANMQAELVAKPSVSVEFVTNMGIIIPDFARSGVQMNTNFFHESGLEAHVALKAGKLKFIIPSPKRPVKLLSGGNTLHLVSTTKTEVIPPLIENRQSWSVCKQVFPGLNYCTSGAYSNASSTDSASYYPLTGDTRLELELRPTGEIEQYSVSATYELQREDRALVDTLKFVTQAEGAKQTEATMTFKYNRQSMTLSSEVQIPDFDVDLGTILRVNDESTEGKTSYRLTLDIQNKKITEVALMGHLSCDTKEERKIKGVISIPRLQAEARSEILAHWSPAKLLLQMDSSATAYGSTVSKRVAWHYDEEKIEFEWNTGTNVDTKKMTSNFPVDLSDYPKSLHMYANRLLDHRVPQTDMTFRHVGSKLIVAMSSWLQKASGSLPYTQTLQDHLNSLKEFNLQNMGLPDFHIPENLFLKSDGRVKYTLNKN",
    contigs="A500-540/0 15-25",  # 20 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=30, 
    num_seq_per_target=1 
)
cycle1D = ExampleRequestParams(
    target_sequence="MDPPRPALLALLALPALLLLLLAGARAEEEMLENVSLVCPKDATRFKHLRKYTYNYEAESSSGVPGTADSRSATRINCKVELEVPQLCSFILKTSQCTLKEVYGFNPEGKALLKKTKNSEEFAAAMSRYELKLAIPEGKQVFLYPEKDEPTYILNIKRGIISALLVPPETEEAKQVLFLDTVYGNCSTHFTVKTRKGNVATEISTERDLGQCDRFKPIRTGISPLALIKGMTRPLSTLISSSQSCQYTLDAKRKHVAEAICKEQHLFLPFSYKNKYGMVAQVTQTLKLEDTPKINSRFFGEGTKKMGLAFESTKSTSPPKQAEAVLKTLQELKKLTISEQNIQRANLFNKLVTELRGLSDEAVTSLLPQLIEVSSPITLQALVQCGQPQCSTHILQWLKRVHANPLLIDVVTYLVALIPEPSAQQLREIFNMARDQRSRATLYALSHAVNNYHKTNPTGTQELLDIANYLMEQIQDDCTGDEDYTYLILRVIGNMGQTMEQLTPELKSSILKCVQSTKPSLMIQKAAIQALRKMEPKDKDQEVLLQTFLDDASPGDKRLAAYLMLMRSPSQADINKIVQILPWEQNEQVKNFVASHIANILNSEELDIQDLKKLVKEALKESQLPTVMDFRKFSRNYQLYKSVSLPSLDPASAKIEGNLIFDPNNYLPKESMLKTTLTAFGFASADLIEIGLEGKGFEPTLEALFGKQGFFPDSVNKALYWVNGQVPDGVSKVLVDHFGYTKDDKHEQDMVNGIMLSVEKLIKDLKSKEVPEARAYLRILGEELGFASLHDLQLLGKLLLMGARTLQGIPQMIGEVIRKGSKNDFFLHYIFMENAFELPTGAGLQLQISSSGVIAPGAKAGVKLEVANMQAELVAKPSVSVEFVTNMGIIIPDFARSGVQMNTNFFHESGLEAHVALKAGKLKFIIPSPKRPVKLLSGGNTLHLVSTTKTEVIPPLIENRQSWSVCKQVFPGLNYCTSGAYSNASSTDSASYYPLTGDTRLELELRPTGEIEQYSVSATYELQREDRALVDTLKFVTQAEGAKQTEATMTFKYNRQSMTLSSEVQIPDFDVDLGTILRVNDESTEGKTSYRLTLDIQNKKITEVALMGHLSCDTKEERKIKGVISIPRLQAEARSEILAHWSPAKLLLQMDSSATAYGSTVSKRVAWHYDEEKIEFEWNTGTNVDTKKMTSNFPVDLSDYPKSLHMYANRLLDHRVPQTDMTFRHVGSKLIVAMSSWLQKASGSLPYTQTLQDHLNSLKEFNLQNMGLPDFHIPENLFLKSDGRVKYTLNKN",
    contigs="A550-590/0 15-25",  # 13 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=20, 
    num_seq_per_target=1 
)

###### Cycle 2: second-half of the ApoB protein ######
cycle2A = ExampleRequestParams(
    target_sequence="GKIDFLNNYALFLSPSAQQASWQVSARFNQYKYNQNFSAGNNENIMEAHVGINGEANLDFLNIPLTIPEMRLPYTIITTPPLKDFSLWEKTGLKEFLKTTKQSFDLSVKAQYKKNKHRHSITNPLAVLCEFISQSIKSFDRHFEKNRNNALDFVTKSYNETKIKFDKYKAEKSHDELPRTFQIPGYTVPVVNVEVSPFTIEMSAFGYVFPKAVSMPSFSILGSDVRVPSYTLILPSLELPVLHVPRNLKLSLPDFKELCTISHIFIPAMGNITYDFSFKSSVITLNTNAELFNQSDIVAHLLSSSSSVIDALQYKLEGTTRLTRKRGLKLATALSLSNKFVEGSHNSTVSLTTKNMEVSVATTTKAQIPILRMNFKQELNGNTKSKPTVSSSMEFKYDFNSSMLYSTAKGAVDHKLSLESLTSYFSIESSTKGDVKGSVLSREYSGTIASEANTYLNSKSTRSSVKLQGTSKIDDIWNLEVKENFAGEATLQRIYSLWEHSTKNHLQLEGLFFTNGEHTSKATLELSPWQMSALVQVHASQPSSFHDFPDLGQEVALNANTKNQKIRWKNEVRIHSGSFQSQVELSNDQEKAHLDIAGSLEGHLRFLKNIILPVYDKSLWDFLKLDVTTSIGRRQHLRVSTAFVYTKNPNGYSFSIPVKVLADKFIIPGLKLNDLNSVLVMPTFHVPFTDLQVPSCKLDFREIQIYKKLRTSSFALNLPTLPEVKFPEVDVLTKYSQPEDSLIPFFEITVPESQLTVSQFTLPKSVSDGIAALDLNAVANKIADFELPTIIVPEQTIEIPSIKFSVPAGIVIPSFQALTARFEVDSPVYNATWSASLKNKADYVETVLDSTCSSTVQFLEYELNVLGTHKIEDGTLASKTKGTFAHRDFSAEYEEDGKYEGLQEWEGKAHLNIKSPAFTDLHLRYQKDKKGISTSAASPAVGTVGMDMDEDDDFSKWNFYYSPQSSPDKKLTIFKTELRVRESDEETQIKVNWEEEAASGLLTSLKDNVPKATGVLYDYVNKYHWEHTGLTLREVSSKLRRNLQNNAEWVYQGAIRQIDDIDVRFQKAASGTTGTYQEWKDKAQNLYQELLTQEGQASFQGLKDNVFDGLVRVTQEFHMKVKHLIDSLIDFLNFPRFQFPGKPGIYTREELCTMFIREVGTVLSQVYSKVHNGSEILFSYFQDLVITLPFELRKHKLIDV",
    contigs="A100-140/0 15-25",  # 20 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=25, 
    num_seq_per_target=1 
)
cycle2B = ExampleRequestParams(
    target_sequence="GKIDFLNNYALFLSPSAQQASWQVSARFNQYKYNQNFSAGNNENIMEAHVGINGEANLDFLNIPLTIPEMRLPYTIITTPPLKDFSLWEKTGLKEFLKTTKQSFDLSVKAQYKKNKHRHSITNPLAVLCEFISQSIKSFDRHFEKNRNNALDFVTKSYNETKIKFDKYKAEKSHDELPRTFQIPGYTVPVVNVEVSPFTIEMSAFGYVFPKAVSMPSFSILGSDVRVPSYTLILPSLELPVLHVPRNLKLSLPDFKELCTISHIFIPAMGNITYDFSFKSSVITLNTNAELFNQSDIVAHLLSSSSSVIDALQYKLEGTTRLTRKRGLKLATALSLSNKFVEGSHNSTVSLTTKNMEVSVATTTKAQIPILRMNFKQELNGNTKSKPTVSSSMEFKYDFNSSMLYSTAKGAVDHKLSLESLTSYFSIESSTKGDVKGSVLSREYSGTIASEANTYLNSKSTRSSVKLQGTSKIDDIWNLEVKENFAGEATLQRIYSLWEHSTKNHLQLEGLFFTNGEHTSKATLELSPWQMSALVQVHASQPSSFHDFPDLGQEVALNANTKNQKIRWKNEVRIHSGSFQSQVELSNDQEKAHLDIAGSLEGHLRFLKNIILPVYDKSLWDFLKLDVTTSIGRRQHLRVSTAFVYTKNPNGYSFSIPVKVLADKFIIPGLKLNDLNSVLVMPTFHVPFTDLQVPSCKLDFREIQIYKKLRTSSFALNLPTLPEVKFPEVDVLTKYSQPEDSLIPFFEITVPESQLTVSQFTLPKSVSDGIAALDLNAVANKIADFELPTIIVPEQTIEIPSIKFSVPAGIVIPSFQALTARFEVDSPVYNATWSASLKNKADYVETVLDSTCSSTVQFLEYELNVLGTHKIEDGTLASKTKGTFAHRDFSAEYEEDGKYEGLQEWEGKAHLNIKSPAFTDLHLRYQKDKKGISTSAASPAVGTVGMDMDEDDDFSKWNFYYSPQSSPDKKLTIFKTELRVRESDEETQIKVNWEEEAASGLLTSLKDNVPKATGVLYDYVNKYHWEHTGLTLREVSSKLRRNLQNNAEWVYQGAIRQIDDIDVRFQKAASGTTGTYQEWKDKAQNLYQELLTQEGQASFQGLKDNVFDGLVRVTQEFHMKVKHLIDSLIDFLNFPRFQFPGKPGIYTREELCTMFIREVGTVLSQVYSKVHNGSEILFSYFQDLVITLPFELRKHKLIDV",
    contigs="A150-190/0 15-25",  # 20 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=25, 
    num_seq_per_target=1 
)
cycle2C = ExampleRequestParams(
    target_sequence="GKIDFLNNYALFLSPSAQQASWQVSARFNQYKYNQNFSAGNNENIMEAHVGINGEANLDFLNIPLTIPEMRLPYTIITTPPLKDFSLWEKTGLKEFLKTTKQSFDLSVKAQYKKNKHRHSITNPLAVLCEFISQSIKSFDRHFEKNRNNALDFVTKSYNETKIKFDKYKAEKSHDELPRTFQIPGYTVPVVNVEVSPFTIEMSAFGYVFPKAVSMPSFSILGSDVRVPSYTLILPSLELPVLHVPRNLKLSLPDFKELCTISHIFIPAMGNITYDFSFKSSVITLNTNAELFNQSDIVAHLLSSSSSVIDALQYKLEGTTRLTRKRGLKLATALSLSNKFVEGSHNSTVSLTTKNMEVSVATTTKAQIPILRMNFKQELNGNTKSKPTVSSSMEFKYDFNSSMLYSTAKGAVDHKLSLESLTSYFSIESSTKGDVKGSVLSREYSGTIASEANTYLNSKSTRSSVKLQGTSKIDDIWNLEVKENFAGEATLQRIYSLWEHSTKNHLQLEGLFFTNGEHTSKATLELSPWQMSALVQVHASQPSSFHDFPDLGQEVALNANTKNQKIRWKNEVRIHSGSFQSQVELSNDQEKAHLDIAGSLEGHLRFLKNIILPVYDKSLWDFLKLDVTTSIGRRQHLRVSTAFVYTKNPNGYSFSIPVKVLADKFIIPGLKLNDLNSVLVMPTFHVPFTDLQVPSCKLDFREIQIYKKLRTSSFALNLPTLPEVKFPEVDVLTKYSQPEDSLIPFFEITVPESQLTVSQFTLPKSVSDGIAALDLNAVANKIADFELPTIIVPEQTIEIPSIKFSVPAGIVIPSFQALTARFEVDSPVYNATWSASLKNKADYVETVLDSTCSSTVQFLEYELNVLGTHKIEDGTLASKTKGTFAHRDFSAEYEEDGKYEGLQEWEGKAHLNIKSPAFTDLHLRYQKDKKGISTSAASPAVGTVGMDMDEDDDFSKWNFYYSPQSSPDKKLTIFKTELRVRESDEETQIKVNWEEEAASGLLTSLKDNVPKATGVLYDYVNKYHWEHTGLTLREVSSKLRRNLQNNAEWVYQGAIRQIDDIDVRFQKAASGTTGTYQEWKDKAQNLYQELLTQEGQASFQGLKDNVFDGLVRVTQEFHMKVKHLIDSLIDFLNFPRFQFPGKPGIYTREELCTMFIREVGTVLSQVYSKVHNGSEILFSYFQDLVITLPFELRKHKLIDV",
    contigs="A200-240/0 15-25",  # 20 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=25, 
    num_seq_per_target=1 
)
cycle2D = ExampleRequestParams(
    target_sequence="GKIDFLNNYALFLSPSAQQASWQVSARFNQYKYNQNFSAGNNENIMEAHVGINGEANLDFLNIPLTIPEMRLPYTIITTPPLKDFSLWEKTGLKEFLKTTKQSFDLSVKAQYKKNKHRHSITNPLAVLCEFISQSIKSFDRHFEKNRNNALDFVTKSYNETKIKFDKYKAEKSHDELPRTFQIPGYTVPVVNVEVSPFTIEMSAFGYVFPKAVSMPSFSILGSDVRVPSYTLILPSLELPVLHVPRNLKLSLPDFKELCTISHIFIPAMGNITYDFSFKSSVITLNTNAELFNQSDIVAHLLSSSSSVIDALQYKLEGTTRLTRKRGLKLATALSLSNKFVEGSHNSTVSLTTKNMEVSVATTTKAQIPILRMNFKQELNGNTKSKPTVSSSMEFKYDFNSSMLYSTAKGAVDHKLSLESLTSYFSIESSTKGDVKGSVLSREYSGTIASEANTYLNSKSTRSSVKLQGTSKIDDIWNLEVKENFAGEATLQRIYSLWEHSTKNHLQLEGLFFTNGEHTSKATLELSPWQMSALVQVHASQPSSFHDFPDLGQEVALNANTKNQKIRWKNEVRIHSGSFQSQVELSNDQEKAHLDIAGSLEGHLRFLKNIILPVYDKSLWDFLKLDVTTSIGRRQHLRVSTAFVYTKNPNGYSFSIPVKVLADKFIIPGLKLNDLNSVLVMPTFHVPFTDLQVPSCKLDFREIQIYKKLRTSSFALNLPTLPEVKFPEVDVLTKYSQPEDSLIPFFEITVPESQLTVSQFTLPKSVSDGIAALDLNAVANKIADFELPTIIVPEQTIEIPSIKFSVPAGIVIPSFQALTARFEVDSPVYNATWSASLKNKADYVETVLDSTCSSTVQFLEYELNVLGTHKIEDGTLASKTKGTFAHRDFSAEYEEDGKYEGLQEWEGKAHLNIKSPAFTDLHLRYQKDKKGISTSAASPAVGTVGMDMDEDDDFSKWNFYYSPQSSPDKKLTIFKTELRVRESDEETQIKVNWEEEAASGLLTSLKDNVPKATGVLYDYVNKYHWEHTGLTLREVSSKLRRNLQNNAEWVYQGAIRQIDDIDVRFQKAASGTTGTYQEWKDKAQNLYQELLTQEGQASFQGLKDNVFDGLVRVTQEFHMKVKHLIDSLIDFLNFPRFQFPGKPGIYTREELCTMFIREVGTVLSQVYSKVHNGSEILFSYFQDLVITLPFELRKHKLIDV",
    contigs="A250-290/0 15-25",  # 20 AA long
    hotspot_res=[], 
    input_pdb_chains=[],
    ca_only=False, 
    use_soluble_model=True,
    sampling_temp=[0.2], 
    diffusion_steps=25, 
    num_seq_per_target=1 
)

In [ ]:
# MODIFY: switch example inputs
example = cycle1A
name = "cycle1A"
root = "/home/ubuntu/nvidia-workbench"


### 1. AlphaFold2 (pre-computed on Colab)

AlphaFold2 is a deep learning model for predicting protein structure from amino acid sequence that has achieved state-of-the-art performance.

**Inputs**:
- `sequence`: amino acid sequence
- `algorithm`: for Multiple Sequence Alignment (MSA), can be either of `jackhmmer` or `mmseqs2`. MMSeqs2 is significantly faster.

**Outputs**:
- A list of predicted structures in PDB format.

**Runtime**: most time consuming!
- 25 minutes for a ~550AA length target_sequence on 1 A6000 GPU; 12 minutes on H100

## 2. RFDiffusion

RFDiffusion applies generative diffusion techniques to create novel protein structures. It excels in designing complex protein architectures, including binders and symmetric assemblies, by sculpting atomic clouds into functional proteins. This step is also available on Colab ([diffusion.ipynb](https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/rf/examples/diffusion.ipynb#scrollTo=TuRUfQJZ4vkM)).

**Inputs**
- `input_pdb` is the protein target in PDB format
- `contigs` is the RFDiffusion language for how to specify regions to work on. See the official [RFDiffusion repo](https://github.com/RosettaCommons/RFdiffusion?tab=readme-ov-file#running-the-diffusion-script) under 'Binder Design' for a full breakdown. A20-60/0 50-100 means to generate a binder to chain A residue 20-60, where the binder is 50-100 residues long. The /0 specifies a chain break.
- `hotspot_res` hot spot residues (specifically for binders)
- `diffusion_steps` number of diffusion_steps

**Output**:
- `output_pdb` is the output pdb;`protein` is the input pdb

**Runtime**: ~15 seconds to 1 minute
- H100 runtime: 9 seconds (for ~550 AA length target_sequence)


In [ ]:
# modified to accept a precomputed PDB structure
precomputed_pdb_path = "cycle1_alphafold2_output.pdb" # converts pdb_id to a Path Object; Reads the entire content of the PDB file as a single string containing only the "ATOM" lines
precomputed_pdb_path = "cycle2_alphafold2_output.pdb" # converts pdb_id to a Path Object; Reads the entire content of the PDB file as a single string containing only the "ATOM" lines

# Run model
precomputed_pdb=get_reduced_pdb(precomputed_pdb_path, rcsb_path=None)
rfdiffusion_query = {
    "input_pdb": precomputed_pdb,  # Now using the precomputed PDB structure
    "contigs": example.contigs,
    "diffusion_steps": example.diffusion_steps
    # "hotspot_res": example.hotspot_res
}
rc, rfdiffusion_response = query_nim(
    payload=rfdiffusion_query,
    nim_endpoint=NIM_ENDPOINTS.RFDIFFUSION.value,
    nim_port=NIM_PORTS.RFDIFFUSION_PORT.value
)
## Print the first 160 characters of the RFDiffusion PDB output
print(rfdiffusion_response["output_pdb"][0:160])
with open(f"{root}/2_rfdiffusion_{name}.pdb", "w") as pdb_file:
    pdb_file.write(rfdiffusion_response["output_pdb"])

## 3. ProteinMPNN
ProteinMPNN (Protein Message Passing Neural Network) is a deep learning-based graph neural network that predicts amino acid sequences for given protein backbones, leveraging evolutionary, functional, and structural information to generate sequences that are likely to fold into the desired 3D structures. The task is to find AA sequence that results in a desired protein structure that is stable and functional.

**Inputs**: 
- `input_pdb` Input protein for which amino acid sequences need to be predicted
- `ca_only` Defaults to false, CA-only model addresses specific needs in protein design where focusing on the alpha carbon (CA)
- `use_soluble_model` soluble vs non-soluble for membrane protein studies
- `num_seq_per_target` defaults to 1. Number of seqs to generate that will fold into the given target protein structure
- `sampling_temp` ranges from 0 to 1 ranges from 0 to 1 and controls the diversity of design outcomes by adjusting the probability values for the 20 amino acids at each sequence position. 
 
**Outputs**:
- `ProteinMPNN.fa` (string): fasta file containing the designed sequence(s) for the given structure.
- `scores`: (array) log-probabilities of the designed sequences, which indicate the likelihood of each sequence given the input structure
- `probs`: (array) predicted probabilities for each amino acid at each position

**Runtime**: < 30 seconds for 20 short sequences
- H100: 8 seconds

In [ ]:
proteinmpnn_query = {
        "input_pdb" : rfdiffusion_response["output_pdb"],
        "input_pdb_chains" : example.input_pdb_chains,
        "ca_only" : example.ca_only,
        "use_soluble_model" : example.use_soluble_model,
        "num_seq_per_target" : example.num_seq_per_target,
        "sampling_temp" : example.sampling_temp
}
rc, proteinmpnn_response = query_nim(
    payload=proteinmpnn_query,
    nim_endpoint=NIM_ENDPOINTS.PROTEINMPNN.value,
    nim_port=NIM_PORTS.PROTEINMPNN_PORT.value
)

Extract FASTA sequences from the output FASTA file created by ProteinMPNN. Then, we'll create binder-target pairs that we can feed to AlphaFold2-Multimer to predict the binder-target complex structure.

In [ ]:
fasta_sequences = [x.strip() for x in proteinmpnn_response["mfasta"].split("\n") if '>' not in x][2:]
binder_target_pairs = [[binder, example.target_sequence] for binder in fasta_sequences]

# Save binder_target_pairs to a .txt file
with open(f"{root}/3_proteinmpnn_target_{name}.txt", "w") as txt_file:
    for i, pair in enumerate(binder_target_pairs):
        binder, target = pair
        txt_file.write(f"Target: {target}\n")
        txt_file.write("\n")  # Add a blank line between pairs for readability

# Save proteinmpnn_response["mfasta"] to a .fasta file
with open(f"{root}/3_proteinmpnn_{name}.fasta", "w") as fasta_file:
    fasta_file.write(proteinmpnn_response["mfasta"])

# Extract scores/probs array
scores = proteinmpnn_response["scores"]
probs = proteinmpnn_response["probs"]

# Save scores and probs to files
with open(f"{root}/3_proteinmpnn_scores_{name}.txt", "w") as scores_file:
    for i, score in enumerate(scores):
        scores_file.write(f"Sequence {i+1}: Score = {score}\n")

with open(f"{root}/3_proteinmpnn_probs_{name}.txt", "w") as probs_file:
    for i, prob_matrix in enumerate(probs):
        probs_file.write(f"Sequence {i+1}:\n")
        for position_probs in prob_matrix:
            probs_file.write(",".join(map(str, position_probs)) + "\n")
        probs_file.write("\n")

### 4. AlphaFold2-Multimer

AlphaFold2-Multimer is a deep learning model that extends the AlphaFold2 pipelines to predict the combined structure a list of input peptide sequences. By default the multimer system will run 5 seeds per model (25 total predictions) for a small drop in accuracy you may wish to run a single seed per model. 

**Inputs**:
- `sequences`: A list of peptide sequences. For this use case, a single pair of sequences (one peptide chain from the ProteinMPNN result plus the original protein sequence used as input to this workflow).
- `algorithm`: The algorithm uses for Multiple Sequence Alignment (MSA). This can be either `jackhmmer` or `mmseqs2`. MMSeqs2 is significantly faster.

**Output**:
- A list of lists of predicted structures in PDB format. A list of five predictions is returned for each input binder-target pair.

**Runtime**:
- Expected runtime: 20 min per binder-target pair.
- Total runtime: roughly 3 hours

In [ ]:
n_processed = 0
multimer_response_codes = [0 for i in binder_target_pairs]
multimer_results = [None for i in binder_target_pairs]

## NOTE: change this value to process more or fewer target-binder pairs.
pairs_to_process = 1

for binder_target_pair in binder_target_pairs:
    multimer_query = {
        "sequences" : binder_target_pair,
        "selected_models" : [1]
    }
    print(f"Processing pair number {n_processed+1} of {len(binder_target_pairs)}")
    rc, multimer_response = query_nim(
        payload=multimer_query,
        nim_endpoint=NIM_ENDPOINTS.AF2_MULTIMER.value,
        nim_port=NIM_PORTS.AF2_MULTIMER_PORT.value
    )
    multimer_response_codes[n_processed] = rc
    multimer_results[n_processed] = multimer_response
    print(f"Finished binder-target pair number {n_processed+1} of {len(binder_target_pairs)}")
    n_processed += 1
    if n_processed >= pairs_to_process:
        break

In [ ]:
## Print just the first 160 characters of the first multimer response
result_idx = 0
prediction_idx = 0
print(multimer_results[result_idx][prediction_idx][0:160])

# Save all AlphaFold-Multimer results to a .txt file
with open(f"{root}/4_multimer_{name}.txt", "w") as results_file:
    for i, result in enumerate(multimer_results):
        results_file.write(f"Result {i+1}:\n")  # Add a header for each result
        if result is not None:  # Check if the result exists
            for prediction_idx, prediction in enumerate(result):
                results_file.write(f"Prediction {prediction_idx+1}:\n")
                results_file.write(prediction)
                results_file.write("\n\n")  # Add spacing between predictions
        else:
            results_file.write("No result available for this pair.\n")
        results_file.write("-" * 50 + "\n")  # Separator between results


# Section D: Generate final output 
### Assessing the predicted binders and structures

Binder-target pair structure refers to a predicted 3D model of a complex formed by two interacting molecules: a binder protein/peptide and a target protein

There are many metrics that can be used to assess the quality of the predicted binder-target structure. The predicted local distance difference test (pLDDT) is a measure of per-residue confidence in the local structure. It has a range of zero to one hundred, with higher scores considered more accurate.

Here, we rank the results of the binder-target pair AlphaFold2-Multimer predictions by their pLDDT.

**Outputs**:
- `pLDDT` (predicted local distance difference test): Per-residue confidence score indicating the reliability of the prediction (higher is better, max 100).  Residues with pLDDT > 90 are high confidence scores for both backbone and sidechain residues, pLDDT values between 70 and 90 are of intermediate confidence and represent accurate backbone prediction, values between 70 and 50 are low confidence, and residues with pLDDT below 50 depict low reliability. 
- `pAE` (Predicted Aligned Error): Indicates the accuracy of the relative positioning of residues in the complex.

In [ ]:
# Function to calculate average pLDDT over all residues 
def calculate_average_pLDDT(pdb_string):
    total_pLDDT = 0.0
    atom_count = 0
    pdb_lines = pdb_string.splitlines()
    for line in pdb_lines:
        # PDB atom records start with "ATOM"
        if line.startswith("ATOM"):
            atom_name = line[12:16].strip() # Extract atom name
            if atom_name == "CA":  # Only consider atoms with name "CA"
                try:
                    # Extract the B-factor value from columns 61-66 (following PDB format specifications)
                    pLDDT = float(line[60:66].strip())
                    total_pLDDT += pLDDT
                    atom_count += 1
                except ValueError:
                    pass  # Skip lines where B-factor can't be parsed as a float

    if atom_count == 0:
        return 0.0  # Return 0 if no N atoms were found

    average_pLDDT = total_pLDDT / atom_count
    return average_pLDDT


In [53]:
plddts = []
for idx in range(0, len(multimer_results)):
    if multimer_results[idx] is not None:
        plddts.append(calculate_average_pLDDT(multimer_results[idx][0]))

In [ ]:
## Combine the results with their pLDDTs
binder_target_results = list(zip(binder_target_pairs, multimer_results, plddts))

## Sort the results by plddt
sorted_binder_target_results = sorted(binder_target_results, key=lambda x : x[2])

## print the top 5 results
for i in range(0, len(sorted_binder_target_results)):
    print("-"*80)
    print(f"rank: {i}")
    print(f"binder: {sorted_binder_target_results[i][0][0]}")
    print(f"target: {sorted_binder_target_results[i][0][1]}")
    print(f"pLDDT: {sorted_binder_target_results[i][2]}")
    print("-"*80)

In [ ]:
# Save all results to a .txt file
with open(f"4_multimer_analysis_{name}.txt", "w") as results_file:
    # Section 1: Average pLDDT values
    results_file.write("### Average pLDDT Values ###\n")
    for idx, plddt in enumerate(plddts):
        results_file.write(f"Result {idx + 1}: Average pLDDT = {plddt:.2f}\n")
    results_file.write("\n\n")

    # Section 2: Binder-Target Results with Raw Data
    results_file.write("### Binder-Target Results (Raw Data) ###\n")
    for idx, (pair, result, plddt) in enumerate(binder_target_results):
        results_file.write(f"Result {idx + 1}:\n")
        results_file.write(f"Binder: {pair[0]}\n")
        results_file.write(f"Target: {pair[1]}\n")
        results_file.write(f"Average pLDDT: {plddt:.2f}\n")
        results_file.write(f"Raw Multimer Result: {result}\n")
        results_file.write("-" * 80 + "\n")
    results_file.write("\n\n")

    # Section 3: Sorted Binder-Target Results by pLDDT
    results_file.write("### Sorted Binder-Target Results by pLDDT ###\n")
    for rank, (pair, result, plddt) in enumerate(sorted_binder_target_results):
        results_file.write(f"Rank {rank + 1}:\n")
        results_file.write(f"Binder: {pair[0]}\n")
        results_file.write(f"Target: {pair[1]}\n")
        results_file.write(f"Average pLDDT: {plddt:.2f}\n")
        results_file.write("-" * 80 + "\n")

print("Saved all AlphaFold-Multimer analysis results to alphafold_multimer_analysis.txt")

These sequences show the highest pLDDT for their binder-target pair.

# Description of outputs

### ProteinMPNN

```
>3HTN, score=1.1705, global_score=1.2045, fixed_chains=['B'], designed_chains=['A', 'C'], model_name=v_48_020, git_hash=015ff820b9b5741ead6ba6795258f35a9c15e94b, seed=37
NMYSYKKIGNKYIVSINNHTEIVKALNAFCKEKGILSGSINGIGAIGELTLRFFNPKTKAYDDKTFREQMEISNLTGNISSMNEQVYLHLHITVGRSDYSALAGHLLSAIQNGAGEFVVEDYSERISRTYNPDLGLNIYDFER/NMYSYKKIGNKYIVSINNHTEIVKALNAFCKEKGILSGSINGIGAIGELTLRFFNPKTKAYDDKTFREQMEISNLTGNISSMNEQVYLHLHITVGRSDYSALAGHLLSAIQNGAGEFVVEDYSERISRTYNPDLGLNIYDFER
>T=0.1, sample=1, score=0.7291, global_score=0.9330, seq_recovery=0.5736
NMYSYKKIGNKYIVSINNHTEIVKALKKFCEEKNIKSGSVNGIGSIGSVTLKFYNLETKEEELKTFNANFEISNLTGFISMHDNKVFLDLHITIGDENFSALAGHLVSAVVNGTCELIVEDFNELVSTKYNEELGLWLLDFEK/NMYSYKKIGNKYIVSINNHTDIVTAIKKFCEDKKIKSGTINGIGQVKEVTLEFRNFETGEKEEKTFKKQFTISNLTGFISTKDGKVFLDLHITFGDENFSALAGHLISAIVDGKCELIIEDYNEEINVKYNEELGLYLLDFNK
>T=0.1, sample=2, score=0.7414, global_score=0.9355, seq_recovery=0.6075
NMYKYKKIGNKYIVSINNHTEIVKAIKEFCKEKNIKSGTINGIGQVGKVTLRFYNPETKEYTEKTFNDNFEISNLTGFISTYKNEVFLHLHITFGKSDFSALAGHLLSAIVNGICELIVEDFKENLSMKYDEKTGLYLLDFEK/NMYKYKKIGNKYVVSINNHTEIVEALKAFCEDKKIKSGTVNGIGQVSKVTLKFFNIETKESKEKTFNKNFEISNLTGFISEINGEVFLHLHITIGDENFSALAGHLLSAVVNGEAILIVEDYKEKVNRKYNEELGLNLLDFNL
```
* `score` - average over residues that were designed negative log probability of sampled amino acids
* `global score` - average over all residues in all chains negative log probability of sampled/fixed amino acids
* `fixed_chains` - chains that were not designed (fixed)
* `designed_chains` - chains that were redesigned
* `model_name/CA_model_name` - model name that was used to generate results, e.g. `v_48_020`
* `git_hash` - github version that was used to generate outputs
* `seed` - random seed
* `T=0.1` - temperature equal to 0.1 was used to sample sequences
* `sample` - sequence sample number 1, 2, 3...etc
-----------------------------------------------------------------------------------------------------

### AlphaFold2-multimer

```
output file
```
* `variable` - description

-----------------------------------------------------------------------------------------------------
```

In [ ]:
# Example outputs